# Chemical Genetic Traces by Drug Class
Demonstration notebook from Jake's lab meeting. June 10, 2020

### Import the python libraries

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive, interact
from IPython.display import display

### Load the chemical-genetic trace data

In [2]:
# Screen_Dict contains the growth measurements
with open('Library/Screen_Dict.pickle', 'rb') as Pickle_File_Handle:
    Screen_Dict  = pickle.load(Pickle_File_Handle)
    
# Full_Library_DF contains information about the Prestwick library
Full_Library_DF = pd.read_csv('Library/Prestwick Library.csv', index_col=[0, 1, 2])

# Drug_Classes_DF assigns drugs to interesting groups
Drug_Classes_DF = pd.read_csv('Library/Clinical Effects.csv', index_col=[0]) 

### Collect growth measurements for the entire library

In [3]:
# Collect all the OD traces
All_Trace_Treatments = ['Arabinose 1E3', 'Arabinose 1E4', 'Arabinose 1E5',
                        'Arabinose 1E6', 'Arabinose 1E7']

All_Traces = {}
All_Traces['OD600 Mean'] = np.vstack([Screen_Dict['All Plates'][Trace_Treatment]['OD600 Mean'] \
                          for Trace_Treatment in All_Trace_Treatments])

Drug_Traces = {}
Drug_Traces['OD600 Mean'] = np.vstack([Screen_Dict['Drug'][Trace_Treatment]['OD600 Mean'] \
                          for Trace_Treatment in All_Trace_Treatments])

### Collect growth measurements for each class

In [4]:
# Connect each drug name to its index in the data
Drug_Position_Index = pd.Series(Full_Library_DF.index.get_level_values(0),
                           index=Full_Library_DF['Chemical name'].values,
                           name='Position')

# Extract the growth data for each drug in each set
Class_Traces = {}
All_Class_Names = Drug_Classes_DF.index.values
for Class_Name in All_Class_Names:
    
    Class_Traces[Class_Name] = {}
    Class_Traces[Class_Name]['Drug Names'] = Drug_Classes_DF.loc[Class_Name].dropna()
    Class_Traces[Class_Name]['Drug Positions'] = Drug_Position_Index.loc[Class_Traces[Class_Name]['Drug Names']].values
    Class_Traces[Class_Name]['OD600 Mean'] = All_Traces['OD600 Mean'][:, Class_Traces[Class_Name]['Drug Positions']]


### Plot the interactive chemical-genetic traces

In [5]:
# Assign the plot style
plt.style.use('Library/Trace_Style.mplstyle')

#%% Color metadata
KColors = {'Dark Gray':'#222222', 'Dark Pink':'#B3446C'}

# Make a dropdown to select the interesting drug class
w = widgets.Dropdown(
    options=All_Class_Names,
    value='Nasal Decongestant',
    description='Drug Class:')

def makeplot(Selected_Class):
    ''' Builds the plot interactively for the selected drug class'''
    
    Fig_handle, Ax_handle = plt.subplots(nrows=1, ncols=4, sharex=True, sharey=True)
    
    Ax_handle[0].plot(Drug_Traces['OD600 Mean'],
                      color=KColors['Dark Gray'], alpha=0.05)

    Ax_handle[1].plot(Class_Traces[Selected_Class]['OD600 Mean'],
                      color=KColors['Dark Pink'], alpha=0.5)
    
    Vplot_handle_1 = Ax_handle[2].violinplot(Drug_Traces['OD600 Mean'].transpose(), positions=range(5),
                            vert=True, showextrema=False)
     
    Vplot_handle_2 = Ax_handle[3].violinplot(Class_Traces[Selected_Class]['OD600 Mean'].transpose(), positions=range(5),
                            vert=True, showextrema=False)
    
    # Set color and style of the plots
    for vp in Vplot_handle_1['bodies']:
            vp.set_facecolor(KColors['Dark Gray'])
            vp.set_alpha(0.8)
            
    for vp in Vplot_handle_2['bodies']:
        vp.set_facecolor(KColors['Dark Pink'])
        vp.set_alpha(0.8)
    
    for Ax in Ax_handle:
        Ax.set_yticks((0.1, 0.2, 0.3, 0.4))
        Ax.set_ylabel('Growth (OD600)')
        Ax.set_xticks([0, 1, 2, 3, 4])
        Ax.set_xticklabels(('$10^3$', '$10^4$', '$10^5$', '$10^6$', '$10^7$'))
        Ax.set_xlabel('Induction Level (mM)')
        
    Ax_handle[0].set_title('All Drugs')
    Ax_handle[1].set_title(Selected_Class)
    Ax_handle[2].set_title('All Drugs')
    Ax_handle[3].set_title(Selected_Class)

interactive(makeplot, Selected_Class=w)

interactive(children=(Dropdown(description='Drug Class:', index=39, options=('Antineoplastic', 'Antipsoriatic'…